Inspiration taken from: https://mccormickml.com/2021/06/29/combining-categorical-numerical-features-with-bert/

In [ ]:
import pandas as pd
import string
import matplotlib.pyplot as plt
import math


from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import transformers
from transformers import BertTokenizer, BertModel
import tensorflow as tf

import torch
from torch.nn.utils.rnn import pad_sequence
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, TensorDataset

In [ ]:
pd.reset_option('display.max_rows')

In [ ]:
df = pd.read_csv('/kaggle/input/spell-checked-df-csv/spell_checked_df.csv')
df

In [ ]:
def remove_words(commentary):
    
    string_value = commentary.replace('[', '').replace(']', '').replace(';', ',')
    string_value = string_value.replace("'", "")
    list_value = string_value.split(',')
    
    words_to_remove = ["sri", "lanka", "hong", "kong", "rajapaksa"]
    return [word.strip() for word in list_value if word.strip() not in words_to_remove]

In [ ]:
df['new_preprocessed_commentary'] = df['preprocessed_commentary'].apply(lambda x: remove_words(x))
df

In [ ]:
def get_batsmen_bowler_names(df):
    bowlers = list(df['bowler'].unique())
    batsmen = list(df['batsman'].unique())
    players = bowlers + batsmen
    players = [name.lower().split()[0] for name in players]
    
    return players

def remove_players(text, players):
    keywords = ['bhuvneshwar', 'suryakumar', 'asalanka', 'silva', 'khushdil', 'kohl', 'nissanka',
                'madushanka', 'iftikhar', 'hasnain', 'hasaranga', 'karunaratn', 'nawab']
    
    players = players + keywords
    return [word for word in text if word not in players]


In [ ]:
players = get_batsmen_bowler_names(df)
df['new_preprocessed_commentary_players_removed'] = df['new_preprocessed_commentary'].apply(lambda x: remove_players(x, players))
df

In [ ]:
#Initialize the 'bowling_team' column with 'Other'
df['bowling_team'] = 'Other'

#Iterate over the range of matches
for match_no in range(1, 14):
    # Extract the batting teams for the current match
    team1 = df[(df['match_no'] == match_no) & (df['batting_side'] == 1)]['batting_team'].iloc[0]
    team2 = df[(df['match_no'] == match_no) & (df['batting_side'] == 2)]['batting_team'].iloc[0]

#Assign the opponent team as the bowling team for each occurrence of a team in the batting team column
    df.loc[(df['match_no'] == match_no) & (df['batting_team'] == team1), 'bowling_team'] = team2
    df.loc[(df['match_no'] == match_no) & (df['batting_team'] == team2), 'bowling_team'] = team1

#Print the DataFrame to verify
df.head()

In [ ]:
df['wicket_fall'] = df['runs'].str.contains('\d[w]', regex=True)
print(df['wicket_fall'].value_counts())

In [ ]:
def group_overs(over):
    integer_part = math.floor(over)    
    decimal_part = over - integer_part
    
          
    grouped_over = integer_part + 1
    
    return grouped_over

In [ ]:
df['grouped_overs'] = df['overs'].apply(group_overs)
df.iloc[120]

In [ ]:
df.groupby(by="grouped_overs")['wicket_fall'].sum()

In [ ]:
df_over = pd.DataFrame()

In [ ]:

df_sorted = df.sort_values(by=["match_no", "batting_side", "grouped_overs"])

grouped = df_sorted.groupby(["match_no", "batting_side", "grouped_overs"])

# Initialize lists to store over-by-over data
over_numbers = []
total_runs = []
wicket_fall = []
match_numbers = []
batting_sides = []
batting_teams = []
bowling_teams = []
commentary = []
comment = []
bowlers = []
batsmen = []

# Iterate over groups
for group_key, group_df in grouped:
    
    match_no, batting_side, over = group_key
    runs_in_over = group_df["numerical_runs"].sum()
    wicket = 1 if group_df["wicket_fall"].any() else 0
    batting_team = " ".join(group_df['batting_team'].unique())
    bowling_team = " ".join(group_df['bowling_team'].unique())
    bowler = " ".join(group_df['bowler'].unique())
    batsman = " ,".join(group_df['batsman'].unique())
    
    comment_list = group_df["new_preprocessed_commentary_players_removed"].tolist()
    comment = []  # Initialize comment as an empty list
    [comment.extend(sublist) for sublist in comment_list]
    
    over_numbers.append(over)
    total_runs.append(runs_in_over)
    wicket_fall.append(wicket)
    match_numbers.append(match_no)
    batting_sides.append(batting_side)
    commentary.append(comment)
    bowling_teams.append(bowling_team)
    batting_teams.append(batting_team)
    bowlers.append(bowler)
    batsmen.append(batsman)

# Create a new DataFrame for over-by-over summary
df_over = pd.DataFrame({
    "match_no": match_numbers,
    "batting_side": batting_sides,
    "batting_team": batting_teams,
    "bowling_team": bowling_teams,
    "over": over_numbers,
    "total_runs": total_runs,
    "wicket_fall": wicket_fall,
    "commentary": commentary,
    "bowlers": bowlers,
    "batsmen": batsmen
})

# Reset index
df_over.reset_index(drop=True, inplace=True)

df_over

In [ ]:
def get_winner(row):
    winners = ['AFG', 'IND', 'AFG', 'IND', 'SL', 'PAK', 'SL', 'PAK', 'SL', 'PAK', 'IND', 'SL', 'SL']
    index = row['match_no'] - 1  # Subtract 1 to convert match number to zero-based index
    if 0 <= index < len(winners):
        return winners[index]
    else:
        return 'Unknown'

df_over['winner'] = df_over.apply(get_winner, axis=1)

In [ ]:
def who_won(row):
    if row['batting_team'] == row['winner']:
        return 1
    elif row['bowling_team'] == row['winner']:
        return 0
    
df_over['winner_label'] = df_over.apply(who_won, axis=1)

In [ ]:
df_over['winner'].value_counts()

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Group by batting and bowling team and aggregate total runs
df_grouped = df_over.groupby(['match_no']).agg({'total_runs': 'sum', 'over': 'count'}).reset_index()
df_grouped.rename(columns={'over': 'total_overs'}, inplace=True)

# Separate features and target variable
X = df_grouped[['total_overs']]
y = df_grouped['total_runs']

# Feature scaling for numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split for model evaluation
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2)

# Define and train Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on testing set
y_pred = model.predict(X_test)

# Evaluate model performance using Mean Squared Error and R-squared
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean Squared Error:", mse)
print("R-squared:", r2)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Create index array to keep track of the indices
indices = np.arange(len(X_test))

# Plotting each match separately
plt.figure(figsize=(15, 10))

# Iterate through each match
for i in range(13):
    match_indices = indices[X_test[:,0] == i]  # Get indices of data points for the current match
    print(f"Match {i+1} indices:", match_indices)
    print(f"Match {i+1} X_test values:", X_test[match_indices])
    plt.subplot(3, 5, i + 1)
    plt.scatter(np.arange(len(match_indices)), y_test.iloc[match_indices], color='blue', label='Actual')
    plt.plot(np.arange(len(match_indices)), y_pred[match_indices], color='red', linestyle='--', label='Predicted')
    plt.title(f'Match {i+1}')
    plt.xlabel('Over')
    plt.ylabel('Total Runs')
    plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# Separate features and target variable
X = df_over[['bowling_team', 'batting_team', 'over']]
y = df_over['total_runs']

# Convert 'over' column to string type
X['over'] = X['over'].astype(str)

# Encode categorical features (bowling_team, batting_team)
encoder = OneHotEncoder(sparse=False)
X_encoded = encoder.fit_transform(X[['bowling_team', 'batting_team']])

# Combine encoded features with numerical feature (over)
X_encoded_df = pd.DataFrame(X_encoded, columns=encoder.get_feature_names_out(['bowling_team', 'batting_team']))
X = pd.concat([X[['over']], X_encoded_df], axis=1)

# Feature scaling for numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.drop(columns=['over']))

# Train-test split for model evaluation
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2)

# Define and train SVR model
model = SVR(kernel='rbf', C=100, epsilon=0.1)
model.fit(X_train, y_train)

# Make predictions on testing set
y_pred = model.predict(X_test)

# Evaluate model performance using Mean Squared Error and R-squared
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean Squared Error:", mse)
print("R-squared:", r2)



In [ ]:
# Create a line plot to compare actual and predicted total runs
plt.plot(np.arange(len(y_test)), y_test, label='Actual Total Runs')
plt.plot(np.arange(len(y_test)), y_pred, label='Predicted Total Runs')
plt.xlabel("Index of Test Data")
plt.ylabel("Total Runs")
plt.title("Actual vs Predicted Total Runs")
plt.legend()
plt.show()

In [ ]:
# Perform groupby operation and calculate the sum of 'wicket_fall' for each 'overs'
grouped_data = df_over.groupby(by="over")['wicket_fall'].sum()

# Create a histogram using pandas plotting functionality
grouped_data.plot(kind='bar', figsize=(10, 6), color='skyblue')

# Customize the plot
plt.title('Wickets Fallen per Over')
plt.xlabel('Overs')
plt.ylabel('Number of Wickets')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.gca().yaxis.set_major_locator(plt.MaxNLocator(integer=True))


# Display the plot
plt.show()


# NLP

In [ ]:
# Convert Series objects to NumPy arrays
numerical_runs_array = df_over['total_runs'].values.reshape(-1, 1)
grouped_overs_array = df_over['over'].values.reshape(-1, 1)

# Normalize the arrays to the range [0, 1]
scaler = preprocessing.MinMaxScaler()
numerical_runs_normalized = scaler.fit_transform(numerical_runs_array)
grouped_overs_normalized = scaler.fit_transform(grouped_overs_array)

# Assign normalized values back to DataFrame
df_over['runs_normalized'] = numerical_runs_normalized
df_over['overs_normalized'] = grouped_overs_normalized

df_over

In [ ]:
# Load the pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
over_data = [text for text in df_over['overs_normalized']]
runs_data = [text for text in df_over['runs_normalized']]

In [ ]:
sen_w_feats = []
labels = []

for index, row in df_over.iterrows():   
    combined = ""
    
    combined += "In the Asia Cup Match between {:} and {:} in over {:}, " \
                "{:} scored {:} runs against bowler {:}. There were {:} wickets. "\
                "The live commentary was: {:}".format(row["batting_team"], 
                                                       row["bowling_team"], 
                                                       row["over"],
                                                       row["batsmen"], 
                                                       row['total_runs'],
                                                       row["bowlers"],
                                                       row['wicket_fall'],
                                                       row['commentary'])
    
    # Add the combined text to the list.
    sen_w_feats.append(combined)

    # Record the sample's label.
    labels.append(row['winner_label'])

print('  DONE.')

print('Dataset contains {:,} samples.'.format(len(sen_w_feats)))

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [ ]:
from transformers import BertForSequenceClassification


model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 2, #2 for binary classification.
)

# desc = model.cuda()

In [ ]:
batch_size = 1
learning_rate = 1e-5
epochs = 2

In [ ]:
max_len = 0

for sent in sen_w_feats:

    input_ids = tokenizer.encode(sent, add_special_tokens=True)
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

In [ ]:
#Set max length = 512
max_len=512

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

print('Encoding all reviews in the dataset...')

for sent in sen_w_feats:
    encoded_dict = tokenizer.encode_plus(
                        sent,
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = max_len,
                        truncation = True,
                        padding = 'max_length',
                        return_attention_mask = True,   
                        return_tensors = 'pt',  
                )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

print('DONE.')

In [ ]:
from sklearn.model_selection import train_test_split


# Split data into training and test sets
train_input_ids, test_input_ids, train_attention_masks, test_attention_masks, train_labels, test_labels = train_test_split(
    input_ids, attention_masks, labels, test_size=0.2, random_state=42)

# Further split the training set into training and validation sets
train_input_ids, val_input_ids, train_attention_masks, val_attention_masks, train_labels, val_labels = train_test_split(
    train_input_ids, train_attention_masks, train_labels, test_size=0.2, random_state=42)

# Create TensorDatasets for each split
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
val_dataset = TensorDataset(val_input_ids, val_attention_masks, val_labels)
test_dataset = TensorDataset(test_input_ids, test_attention_masks, test_labels)

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset),
            batch_size = batch_size
        )

validation_dataloader = DataLoader(
            val_dataset,
            sampler = SequentialSampler(val_dataset),
            batch_size = batch_size
        )

In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(),
                  lr = learning_rate, 
                  eps = 1e-8 
                )

In [ ]:
from transformers import get_linear_schedule_with_warmup


total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [ ]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
import random
import time
import torch
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm

# Set the seed for reproducibility
seed_val = 18
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

#training_stats = []

#scaler = torch.cuda.amp.GradScaler(init_scale=2**8)

for epoch_i in range(epochs):
    # Start tqdm progress bar for training
    with tqdm(total=len(train_dataloader), desc=f'Epoch {epoch_i + 1}/{epochs}', unit='batch') as pbar:
        
        total_train_loss = 0
        
        # Measure how long the training epoch takes.
        t0 = time.time()

        # Put the model into training mode.
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            # Clear any previously calculated gradients.
            model.zero_grad()

            with torch.cuda.amp.autocast():
                result = model(b_input_ids, 
                             token_type_ids = None,
                             attention_mask=b_input_mask,
                             labels=b_labels,
                             return_dict=True)

                loss = result.loss
                logits = result.logits

            # Accumulate the training loss over all of the batches.
            total_train_loss += loss.item()

            # Perform a backward pass to calculate the gradients.
            scaler.scale(loss).backward()

            # Clip the norm of the gradients to 1.0.
            #scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and take a step using the computed gradient.
#             scaler.step(optimizer)
#             scaler.update()

            optimizer.step()

            # Update the learning rate.
            scheduler.step()

            # Release GPU memory
            torch.cuda.empty_cache()

            # Update tqdm progress bar
            pbar.update(1)
            pbar.set_postfix({'loss': loss.item()})

        # Calculate the average loss over all of the batches.
        avg_train_loss = total_train_loss / len(train_dataloader)

        # Measure how long this epoch took.
        training_time = time.time() - t0

        print("")
        print("Average training loss: {0:.2f}".format(avg_train_loss))
        print("Training epoch took: {:.2f} seconds".format(training_time))

In [ ]:
# Display floats with two decimal places.
pd.set_option('precision', 2)

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')

# Display the table.
df_stats

In [ ]:
#Model needs to be run on validation_dataloader but the model training with train_dataset not completing due to GPU resource issues 
# even after reducing batch size and attempting to use a scaler